In [ ]:
# Importeer benodigde modules
from IPython.display import display
from pathlib import Path, PurePath
from tqdm.notebook import tqdm
from ultralytics import YOLO
import ipywidgets as widgets
import pandas as pd
import numpy as np
import logging
import shutil
import os
import sys

# Herhaaldelijke Validatie
Dit script is bedoeld om een model te valideren op een volledige dataset. Het script maakt datasets/datapunten op basis van segmentatie, maar wanneer afbeeldingen niet gesegmenteerd zijn of te weinig segmenten en/of bounding boxes bevatten, kan de optie "GebruikVariabelClustering" op True worden gezet. Dit zorgt ervoor dat de datasets worden aangemaakt op basis van de unieke combinatie van variabelen in plaats van segmenten. In dit geval zou "0002_A52_flits_1" bijvoorbeeld in de dataset "A52_Flits_1" komen, in plaats van in "0002_A52_flits_1".
Na het maken van de datasets/datapunten wordt het opgegeven model op deze datasets/datapunten gevalideerd. Dit biedt de gebruiker de mogelijkheid om de effecten van verschillende variabelen te analyseren. De output van het script is een Excel-bestand met gedetailleerde informatie over de prestaties van iedere dataset/datapunt, zoals precisie, F1-score, confusion matrix, enzovoort. Daarnaast is het mogelijk om clusters van verschillende variabelen te maken door de positie van het cluster op te geven via de variabele "Clustering" om de effecten van combinaties van verschillende variabelen te analyseren. Dit betekent dat alle afbeeldingen, naast dat ze in de individuele datasets terechtkomen, ook toegevoegd worden aan de dataset die de opgegeven combinatie van variabelen weerspiegelt. Bijvoorbeeld, een afbeelding met de naam "0002_A52_flits_1_0_0" en een clustering van [2,3] komt niet alleen in de dataset "0002_A52_flits_1" terecht, maar ook in de dataset "Cluster van A52 en flits". Een andere afbeelding, zoals "0002_ip7_flits_1_0_0", komt in de dataset "0002_ip7_flits_1" en "Cluster van ip7 en flits" terecht. 

**LET OP!** Dit script gaat er vanuit dat de bestanden cordinaten ("0002_ip7_flits_1 **_0_0** ") en/of variabelen ("0002 **_ip7_flits_1** _0_0") in de naam hebben. Voor het gemakelijk toeveoegen van variabelen in de naam zie "BestandenHernoemen.ipynb"

---
<blockquote style="border-left: 5px solid red; padding-left: 10px;">

## **Attentie!** 
<code> Update onderstaande cell voordat je de het script runt </code></blockquote>
 

In [ ]:
# Input map met de segmenten en labels.
InputAnnotaties : list[Path] = [Path(r"pad/naar/jouw/folder"), Path(r"pad/naar/jouw/andere/folder")] #segmenten en labels allenbij
# Bijvoorbeeld: [Path(r"pad/naar/jouw/folder"), Path(r"pad/naar/jouw/andere/folder")]

# Output map waar de "Stitched Images en labels" worden opgeslagen.
OutputFolder : Path = Path(r"Nieuwe Map")
# Bijvoorbeeld: Path(r"pad/naar/jouw/output/folder") OF Path(r"Nieuwe Map")

# Locatie van het model dat gebruikt wordt (Moet .onnx of .pt zijn).
ModelLocatie : Path = Path(r"pad/naar/jouw/model/bestand.pt")
# Bijvoorbeeld: Path(r"pad/naar/jouw/model/bestand.pt")

# Validaties met te weinig annotaties/ afbeeldingen zijn onbetrouwbaar. Om dit tegen te gaan bepaal of:
    # FALSE : één datapunt bestaat uit alle segmenten van een afbeelding ("0002_A52_flits_1_0_0" staat in datapunt: "0002_A52_flits")
    # TRUE : één datapunt bestaat uit alle afbeeldingen met die unieke combinatie van variabelen. ("0002_A52_flits_1_0_0" staat in datapunt: "A52_flits")
GebruikVariabelClustering : bool = True
# Bijvoorbeeld: True OF False

# Welk scheidingsteken word gebruikt in de naam tussen de variabelen? Bijv. "0002_A52_flits_1_0_0" ScheidingsTeken = "_"
    # Wanneer de variabele leeg blijft worden er geen extra kolommen gemaakt voor de variabelen
ScheidingsTekenVariabelen : str = "-"
# Bijvoorbeeld: "-" of "_" of "" etc.

# Welke variabele wil je combineren voor een extra variabelen. Bijv. "0002_A52_flits_1_0_0" variabele 2= A52 (Rest van dataset: ip7 of ip13), variabele 3= flits (Rest van dataset: Geen flits)
    # dus met Clustering = [2, 3] is er een tabblad voor alle mogelijke combinaties van variabele [A52, ip7, ip13] met [Flits, Geen flits] in de output aanwezig
Clustering : list[int] = []
# Bijvoorbeeld: [0, 2] OF []

# Voor variabelen met verschillende namen maar die wel als een geclusterd moeten worden. Bijv. "0002_A52_flits_1_0_0" variabele 2= A52 (Rest van dataset: ip7 of ip13)
    # VariabelenMatch = {"Ip" : ["ip7", "ip13"]} zocht er voor dat wanneer een variabele ip7 of ip13 is deze in de clustering als "Ip" herkent worden maar in de losse foto's nog wel correct genoteerd staan
    # In de clusters komt dus allen A52 en Ip terug voor variabele 2
VariabelenMatch : dict[str, list[int]] = {} # HOOFDLETTER GEVOELIG!
# Bijvoorbeeld: {"Ip" : ["ip7", "ip13"], "Clusternaam": ["OGVariabel1", "OGVariabel2", "OGVariabel5"]}

---
## **Code**

In [ ]:
# Onderdrukt de printstatements van yolo
logging.getLogger('ultralytics').setLevel(logging.ERROR)

# Standaardiseer benaming van clusters. aanpassingen hier worden in het hele script gebruikt
ClusterSignaal = "Cluster van "
EnSignaal = " en "
labels = ""
ScheidingsTekenCoördinaten = "_"
MinimaalAantalBBoxen = 10
MinimaalAantalBestanden = 5

# aanmaak lege variabelen
PathsInput = []
DuplicaatDocumenten = []
AllTextInput = []
AllImageInput = []
RootFiles = set()
ClusterDictionary ={}

# Voorkomt verwarring door oude data
if os.path.exists(OutputFolder):
    shutil.rmtree(OutputFolder)

# Maak de display widgets aan
OutputScreen = widgets.Output()
OutputScreenLoadingBar = widgets.Output()
OutputScreenPrintDisplay = widgets.Output()
with OutputScreen:
    display(OutputScreenLoadingBar)
    display(OutputScreenPrintDisplay)

# Standaardiseert en centreert het kopiëren en opslaan van bestanden naar de correcte locatie (YOLO Map structuur)
def FileCopy(OutputFolder = OutputFolder, Main : str = None, IsLabel : bool = None, FilePath : Path = None):
    OutputDir = os.path.join(OutputFolder, Main, ("labels" if IsLabel else "images"), "validation")
    if not os.path.exists(OutputDir):
        os.makedirs(OutputDir)
    if not os.path.exists(os.path.join(OutputDir, FilePath.name)):
        shutil.copy2(FilePath, OutputDir)
    else:
        # Maakt een lijst met duplicaat bestanden die later geprint worden. Zo kan de Gebruiker controleren of het skippen terecht was
        DuplicaatDocumenten.append(FilePath)

# Safety checks van de opgegeven data zodat errors beter te vertalen zijn
for dir in InputAnnotaties:
    if not os.path.exists(dir):
        sys.exit("Een of meerdere van de opgegeven mappen in FilePaths kan niet gevonden worden. de code is onderbroken")
if not os.path.exists(ModelLocatie):
    sys.exit("Het opgegeven model kan niet gevonden worden. de code is onderbroken")

# Maakt een lijst van alle bestanden in de input mappen en hun sub mappen. slaan de File.stem op om te controleren of we labels bij afbeeldingen hebben 
# en slaan de File.parent op om alle mappen met bruikbare bestanden te kunnen gebruiken ipv alle folder iedere keer te scannen
with OutputScreenLoadingBar:
    for Paths in tqdm(InputAnnotaties, desc="1/4 Bestanden scannen"):
        for root, dirs, files in os.walk(Paths):
            for File in files:
                File = Path(os.path.join(root, File))
                if File.suffix.lower() == ".txt":
                    AllTextInput.append(File.stem)
                    RootFiles.add(File.parent)
                elif File.suffix.lower() in [".jpg", ".png", "jpeg"]:
                    AllImageInput.append(File.stem)
                    RootFiles.add(File.parent)

if len(set(AllImageInput).intersection(AllTextInput)) == 0:
    sys.exit("Er zijn geen Images met bijbehorende labels gevonden. de code is onderbroken")

# scan alle potentiële bestanden en behoud alleen de path's van labels met afbeeldingen en vice versa
# Is een aparte loop van voorgaande om moeilijk doen met de volgorde van eerst label of afbeelding vinden te omzeilen
with OutputScreenLoadingBar:
    for Paths in tqdm(RootFiles, desc="2/4 Bestanden Selecteren"):
        Scan = PurePath(Paths)
        for File in tqdm(os.listdir(Paths), f"| - {Scan.parent.name}/{Scan.name}", leave=True):
            File = Path(os.path.join(Scan, File))
                
            if not os.path.isdir(File):
                if File.stem in AllImageInput and File.stem in AllTextInput:
                    PathsInput.append(File)

                # Filtert alle "labels.txt" eruit en slaat één versie op. geeft een print statement als het 2 unieke bestanden vind met andere (volgorde van) labels.
                # Dit kan later in het script voor problemen zorgen als de labels echt anders zijn.
                elif File.name.lower() == "labels.txt":
                    if not labels:
                        labels = File
                        with open(labels, "r") as Text:
                            labelsText=Text.read()
                    else:
                        with open(File, "r") as Text:
                            if Text.read() != labelsText:
                                with OutputScreenPrintDisplay:
                                    print(f"de inhoud van labelbestanden op:\n{labels}\n\nverschild met de inhoud van\n{files}\n\nDe inhoud van het volgende bestand word aangehouden:\n{labels}\n\n")

#*************************************************************************************************
# Functie om te kunnen valideren dat een File.stem coördinaten bevat en dus een segment is
def ContainsCords(FileStem : str):
    NameSplit = FileStem.rsplit(ScheidingsTekenCoördinaten, 2)

    try:
        NameSplit[-3]
        int(NameSplit[-2])
        int(NameSplit[-1])
        return True
    except (ValueError, IndexError):
        return False

# Functie om te kunnen valideren dat een File.stem opgedeeld kan worden aan de hand ven het opgegeven scheidingsteken    
def ContainsVariables(FileStem: str):
    if ScheidingsTekenVariabelen:
        NameSplit = FileStem.rsplit(ScheidingsTekenVariabelen)

        # Wanneer het teken van variabelen en coördinaten gelijk is en een naam bevat coördinaten is het minimum delen 3 : naam _ ycor _ xcor 
        if ContainsCords(FileStem) and ScheidingsTekenVariabelen == ScheidingsTekenCoördinaten:
            MinimumLength = 3
        else:
            MinimumLength = 1

        if len(NameSplit) > MinimumLength:
            return True
        else:
            return False
    else:
        return False

Clustering.sort()

# Loop om bestanden te koieeren naar de nieuwe dataset/datapunt
with OutputScreenLoadingBar:
    for files in tqdm(PathsInput, desc="3/4 Bestanden herordenen"):
        File = Path(files)
        FileStem = File.stem

        # Bepaalt de dataset voor opslag. houd rekening mee of de gebruiker segmentatie of variabelen wil gebruiken voor groepering. 
        # Maar negeert instellingen wanneer geen coördinaten gevonden kan worden. Las er ook geen variabelen gevonden kan worden of geen scheiding teken is opgegeven word een bestand niet mee genomen
        # Dit is om validatie op te kleine datasets te voorkomen
        if ContainsCords(FileStem) and not GebruikVariabelClustering:
            MainDir, _, _ = FileStem.rsplit(ScheidingsTekenCoördinaten, 2)
        elif ContainsCords(FileStem) and ContainsVariables(FileStem):
            Tijdelijk, _, _ = FileStem.rsplit(ScheidingsTekenCoördinaten, 2)
            _, MainDir = Tijdelijk.split(ScheidingsTekenVariabelen, 1)
        elif ContainsVariables(FileStem):
            _, MainDir = FileStem.split(ScheidingsTekenVariabelen, 1)
        else:
            with OutputScreenPrintDisplay:
                if GebruikVariabelClustering and ContainsCords(FileStem):
                    print(f"{File.name} mag niet op Segmenten geclusterd worden en bevat geen valide variabelen({ScheidingsTekenVariabelen}). \n Om validatie op datasets met minder dan {MinimaalAantalBestanden} bestand te voorkomen word {File.name} niet veder meegenomen.")
                elif GebruikVariabelClustering:
                    print(f"{File.name} mag niet en kan niet op Segmenten geclusterd worden en bevat geen valide variabelen({ScheidingsTekenVariabelen}). \n Om validatie op datasets met minder dan {MinimaalAantalBestanden} bestand te voorkomen word {File.name} niet veder meegenomen.")
                else:
                    print(f"{File.name} kan niet op Segmenten geclusterd worden en bevat geen valide variabelen({ScheidingsTekenVariabelen}). \n Om validatie op datasets met minder dan {MinimaalAantalBestanden} bestand te voorkomen word {File.name} niet veder meegenomen.")
                continue

        # filterd op bestandstype en slaat bestand dan op in nieuwe dataset/datapunt
        if File.suffix.lower() == ".txt":
            IsLabel = True
            FileCopy(Main = MainDir, 
                    IsLabel = IsLabel,
                    FilePath = File
                    )
        elif File.suffix.lower() in [".jpg", ".png", "jpeg"]:
            IsLabel = False
            FileCopy(Main = MainDir, 
                    IsLabel = IsLabel,
                    FilePath = File
                    )
        
        # Checkt of er clusters gemaakt moeten worden en zo ja of het huidige bestand ook aan deze dataset/dit datapunt toegevoegd moet worden
        if Clustering and ScheidingsTekenVariabelen:
            FileSplit = MainDir.split(ScheidingsTekenVariabelen) 

            # Sorteer opgegeven variabelen om de grootse te kunnen selecteren. 
            # gebruikt om te checken of huidige bestand genoeg variabelen heeft om mee geclusterd te worden
            Clustering.sort()
            if len(FileSplit) >= Clustering[-1]:
                ClusterOpslagNaam = f"{ClusterSignaal}"

                for Variabelen in Clustering:

                    VariabeleText = FileSplit[Variabelen-1] # -1 omdat listitems genummerd worden vanaf 0 ipv 1

                    # Clusterd en veranderd namen van variabelen zoals opgegeven in VariabelenMatch door gebruiker
                    for Key in VariabelenMatch.keys():
                        for String in VariabelenMatch[Key]:
                            if String == VariabeleText:
                                VariabeleText = Key                
                    
                    # Slaat de positie(int) van het variabel in de naam op in een dic. In de output excel worden variabelen opgesplitst in kolommen met als header: Variabele i 
                    # met i de positie in het variabel in de originele bestandsnaam/MainDir omdat niet alle variabelen mee genomen worden in iedere clustering moet deze data terug te vinden zijn.
                    ClusterDictionary[VariabeleText] = Variabelen
                    
                    if ClusterOpslagNaam == ClusterSignaal: # True = eerste variabel die word toegevoegd
                        ClusterOpslagNaam = ClusterOpslagNaam + VariabeleText
                    else:
                        ClusterOpslagNaam = ClusterOpslagNaam + EnSignaal + VariabeleText

                FileCopy(
                    Main = ClusterOpslagNaam,
                    IsLabel=IsLabel,
                    FilePath = File
                )
            else:
                with OutputScreenPrintDisplay:
                    print(f"De hoogste waardes van Clustering (={Clustering[-1]}) zijn groter dan het aantal Variabelen in {MainDir} (={len(FileSplit)}). \n Bestand {File.stem} word niet meegenomen in de clustering")
#*************************************************************************************************                    

# Eerst alle submappen tellen
total_dirs = sum(len(dirs) for _, dirs, _ in os.walk(OutputFolder))

# Voegt de labels en configfile toe aan iedere dataset/ ieder datapunt
with OutputScreenLoadingBar:
    with tqdm(total=total_dirs, desc="4/4 Aanvullende bestanden toevoegen") as pbar:
        for root, dirs, files in os.walk(OutputFolder):
            for dir in dirs:
                if dir == "labels": # labels komt één keer per dataset/datapunt voor dus images word genegeerd om duplicaat bestanden te voorkomen
                    shutil.copy2(labels, os.path.join(root, dir, "validation", "Labels.txt"))


                    with open(labels, "r") as a:
                        names = {i: label.strip() for i, label in enumerate(a.readlines())}

                    #genereert de text die uiteindelijk in het config bestand moet komen
                    yaml_dict = {
                        "path": os.path.join(os.getcwd(), root),
                        "train": "images\\train",
                        "val": "images\\validation",
                        "names": names
                    }

                    #Maakt het een leeg config bestand met de juiste naam aan.
                    Root = Path(root)
                    ConfigLocation = os.path.join(root, f"{Root.name}_Config.yaml")
                    with open(ConfigLocation, "w+") as b:
                        b.write("")
                    
                    #Schrijft het daadwerkelijke config bestand. Maakt een uitzondering voor names omdat deze een aparte notatie moet hebben
                    with open(ConfigLocation, "a") as c:
                        for key, value in yaml_dict.items():
                            if key != "names":
                                c.write(f"{key}: {value}\n")
                            else:
                                c.write(f"\nnames:\n")
                                for k, v in names.items():
                                    c.write(f"  {k}: {v}\n")
                    
                pbar.update(1)  # De voortgang met 1 verhogen bij elke iteratie


with OutputScreenPrintDisplay:
    if DuplicaatDocumenten:
        print("De volgende documenten hebben duplicaten in de input map staan en zijn maar eenmalig gekopieerd:")
        for Files in DuplicaatDocumenten:
            Files = Path(Files)
            print(f"| - {Files.name}")
        print("---" * 20)
#******************************************************************************************************************
# Laat benodigde lege variabelen in
Model = YOLO(ModelLocatie, task='detect')
TotaalPerDataPunt = pd.DataFrame()
MatrixPerDataPunt = pd.DataFrame()
TotaalPerCluster = pd.DataFrame()
MatrixPerCluster = pd.DataFrame()
ConfigCount = 0
MinimaalAantalBBoxen = 10

# Maakt een class aan om hardnekkige print statements van YOLO validatie af te vangen
class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.close()
        sys.stderr.close()
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr

# Bepaalt de lengte ven de laatbalk
for root, dirs, files in os.walk(OutputFolder):
    for File in files:
        if File.endswith("_Config.yaml"):
            ConfigCount += 1



# loopt over alle dataset/datapunten heen en valideert ze een voor een
OutputScreenLoadingBar.clear_output(wait = True)
with OutputScreenLoadingBar:
    with tqdm(total=ConfigCount, desc="Datasets valideren") as pbar:
        for root, dirs, files in os.walk(OutputFolder):
            for File in files:
                if File.endswith("_Config.yaml"):
                    DataSet : str = File.removesuffix("_Config.yaml")
                    UsedLabels = set()
                    BBoxCount = 0
                    BestandCount = 0
                                
                    # Loopt over alle Labels in de dataset/datapunt om aantal labels per uniek label te bepalen.
                    # Word gebruikt om alleen de labels die gebruikt zijn toe te voegen aan de output excel 
                    # Controleert ook of dataset genoeg datapunten bevat voor validatie
                    for filename in os.listdir(os.path.join(root, "labels", "validation")):
                        if filename.endswith('.txt') and filename != "Labels.txt":  # Assuming annotations are stored as text files
                            BestandCount += 1
                            with open(os.path.join(root, "labels", "validation", filename), 'r') as f:
                                lines = f.readlines()
                                for line in lines:
                                    BBoxCount += 1
                                    UsedLabels.add(line.split()[0])
                    if BestandCount < MinimaalAantalBestanden:
                        with OutputScreenPrintDisplay:
                            print(f"Het datapunt {DataSet} heeft maar {BestandCount} Afbeeldingen. Om validatie op datasets met minder dan {MinimaalAantalBestanden} bestand te voorkomen word {DataSet} niet veder meegenomen.")
                        pbar.update(1)
                        continue
                    elif BBoxCount < MinimaalAantalBBoxen:
                        with OutputScreenPrintDisplay:
                            print(f"Het datapunt {DataSet} heeft maar {BBoxCount} Bounding Boxen. Om validatie op datasets met minder dan {MinimaalAantalBBoxen} Bounding Boxen te voorkomen word {DataSet} niet veder meegenomen.")
                        pbar.update(1)
                        continue

                    # valideert zonder printstatements
                    with SuppressOutput():
                        ValResult = Model.val(
                            data=os.path.join(root, File), 
                            save_json=True, 
                            exist_ok=True, 
                            split="val", 
                            project= root, 
                            name= "MassValidatieOutput", 
                            augment=True
                        )

                    Headers = []
                    ValueOutput = True
                    ValueCounter = 0
                    while ValueOutput == True:
                        Head = ValResult.names[ValueCounter]
                        if Head:
                            if str(ValueCounter) in UsedLabels:
                                Headers.append(Head)
                            Head = ""
                            ValueCounter += 1
                        if ValueCounter not in ValResult.names:
                            ValueOutput = False

                    # Verzamel alle resultaatdata van de validatie
                    Headers = np.array(Headers)
                    Precision = ValResult.box.p
                    Recall = ValResult.box.r
                    mAP50 = ValResult.box.ap50
                    mAP95 = ValResult.box.ap
                    F1Score = ValResult.box.f1

                    # maakt output excel met de verzamelde data
                    ExelOutput = pd.DataFrame([Headers, Precision, Recall, F1Score, mAP50, mAP95], index=["class", "Box Precision", "Box Recall", "Box F1-score", "mAP50", "mAP50-95"])
                    ExelOutput = ExelOutput.transpose()

                    # Gebruikt het eerder gevonden Labels.txt om de headers van de nieuwe tabel te bepalen
                    with open(labels, "r") as a:
                        HeadersConfusionMatrix = [x.strip("\n") for x in a.readlines()]
                    HeadersConfusionMatrix.append("Achtergrond")

                    # Haalt de data voor de confusion matrix uit de validatie resultaten en update de opmaak voor het output excel
                    ConfusionMatrix = pd.DataFrame(
                        ValResult.confusion_matrix.matrix,  
                        index = map(lambda x: "Model: "+x, HeadersConfusionMatrix),
                    )
                    ConfusionMatrix = ConfusionMatrix.T

                    RawData = pd.DataFrame(
                        columns = [
                            "class",
                            "Mens: Instances", 
                            "Model: Instances", 
                            "True Positives", 
                            "False Positives", 
                            "False Negatives"
                        ])
                    RawData["class"] = HeadersConfusionMatrix[0:(len(HeadersConfusionMatrix)-1)]
                    
                    # Bereken extra waardes op basis van het samenvatten van resultaten
                    for i in range(0, (len(HeadersConfusionMatrix)-1)):
                        RawData.loc[RawData.index[i], "Mens: Instances"] = ConfusionMatrix.iloc[i].sum()
                        RawData.loc[RawData.index[i], "Model: Instances"] = ConfusionMatrix.iloc[:,i].sum()
                        RawData.loc[RawData.index[i], "True Positives"] = ConfusionMatrix.iloc[i, i]
                        RawData.loc[RawData.index[i], "False Positives"] = ConfusionMatrix.iloc[:,i].sum() - ConfusionMatrix.iloc[i, i]
                        RawData.loc[RawData.index[i], "False Negatives"] = ConfusionMatrix.iloc[i].sum() - ConfusionMatrix.iloc[i, i]


                    ConfusionMatrix.insert(0, "", list(map(lambda x: "Mens: "+x, HeadersConfusionMatrix)))

                    ExelOutput = pd.merge(RawData, ExelOutput, on="class", how = "outer")

                    # Berekend een ongewogen gemiddelde van alle waardes en voegt deze toe aan de output excel
                    MensInstances = ExelOutput["Mens: Instances"].sum()
                    ModelInstances = ExelOutput["Model: Instances"].sum()
                    TruePositives = ExelOutput["True Positives"].sum()
                    FalsePositives = ExelOutput["False Positives"].sum()
                    FalseNegatives = ExelOutput["False Negatives"].sum()
                    PrecisionMean = np.nanmean(Precision)
                    RecallMean = np.nanmean(Recall)
                    mAP50Mean = np.nanmean(mAP50)
                    mAP95Mean = np.nanmean(mAP95)
                    
                    ExelOutput = ExelOutput.T
                    ExelOutput[len(ExelOutput.columns)] = [
                        "Ongewogen gemiddelden",  
                        MensInstances,
                        ModelInstances,
                        TruePositives,
                        FalsePositives,
                        FalseNegatives, 
                        PrecisionMean, 
                        RecallMean, 
                        (2*((PrecisionMean * RecallMean)/(PrecisionMean + RecallMean))), # Formule van de F1 score
                        mAP50Mean, 
                        mAP95Mean, 
                    ]
                    ExelOutput = ExelOutput.T

                    # Berekend P, R en F1 opnieuw volgens de formule ipv het gemiddelde van vele losse waardes te gebruiken
                    ExelOutput.insert(
                        6, 
                        "ConfusionMatrix Precision", 
                        (
                            ExelOutput["True Positives"]/
                            (ExelOutput["True Positives"] + ExelOutput["False Positives"])
                        ))

                    ExelOutput.insert(
                        7, 
                        "ConfusionMatrix Recall", 
                        (
                            ExelOutput["True Positives"]/
                            (ExelOutput["True Positives"] + ExelOutput["False Negatives"])
                        ))

                    ExelOutput.insert(
                        8, 
                        "ConfusionMatrix F1-score", 
                        (
                            (2 * (ExelOutput["ConfusionMatrix Precision"] * ExelOutput["ConfusionMatrix Recall"]))/
                            (ExelOutput["ConfusionMatrix Precision"] + ExelOutput["ConfusionMatrix Recall"])
                            
                        ))

                    # Voegt kolom met de dataset naam toe
                    ExelOutput.insert(0, "SampleName", DataSet)
                    ConfusionMatrix.insert(0, "SampleName", DataSet)
                    Variabele = 1
                    
                    # Voegt de individuele resultaten samen tot een groot excel voor opslag
                    if DataSet.startswith(ClusterSignaal):
                        Clustering.sort(reverse=True)

                        # Voegt Dummy kolommen toe voor alle variabelen. variabelen in dit cluster worden later toegevoegd en overschrijven deze warde. 
                        # Zorcht ervoor dat waardes niet overschreven worden en dat Colomnummer voor specefieke variabelen gelijk zijn tussen clusters en losse data
                        for values in Clustering:
                            ExelOutput.insert(1, f"Variabele {values}", "*")
                            ConfusionMatrix.insert(1, f"Variabele {values}", "*")

                        DataSetVar = DataSet.removeprefix(ClusterSignaal).replace(EnSignaal, ScheidingsTekenVariabelen)

                        # Vervangt Dummy waardes met echte variabelen waar nodig         
                        for Var in DataSetVar.split(ScheidingsTekenVariabelen):
                            ExelOutput[f"Variabele {ClusterDictionary[Var]}"] = Var
                            ConfusionMatrix[f"Variabele {ClusterDictionary[Var]}"] = Var
                            Variabele += 1

                        # Voegt De output van dit cluster toe aan de totale output. Er word rekening gehouden met aantal kolommen omdat ander het overschot niet mee genomen word
                        if len(TotaalPerCluster.columns) >= len(ExelOutput.columns):
                            TotaalPerCluster = pd.concat([TotaalPerCluster, ExelOutput], ignore_index= True)
                            MatrixPerCluster = pd.concat([MatrixPerCluster, ConfusionMatrix], ignore_index= True)
                        else:
                            TotaalPerCluster = pd.concat([ExelOutput, TotaalPerCluster], ignore_index= True)
                            MatrixPerCluster = pd.concat([ConfusionMatrix, MatrixPerCluster], ignore_index= True)

                        pbar.update(1)
                    
                    else:
                        if ScheidingsTekenVariabelen:                        
                            for Var in DataSet.split(ScheidingsTekenVariabelen):
                                ExelOutput.insert(Variabele, f"Variabele {Variabele}", Var)
                                ConfusionMatrix.insert(Variabele, f"Variabele {Variabele}", Var)
                                Variabele += 1

                        # Voegt De output van dit datapunt/dataset toe aan de totale output. Er word rekening gehouden met aantal kolommen omdat ander het overschot niet mee genomen word
                        if len(TotaalPerDataPunt.columns) >= len(ExelOutput.columns):
                            TotaalPerDataPunt = pd.concat([TotaalPerDataPunt, ExelOutput], ignore_index= True)
                            MatrixPerDataPunt = pd.concat([MatrixPerDataPunt, ConfusionMatrix], ignore_index= True)
                        else:
                            TotaalPerDataPunt = pd.concat([ExelOutput, TotaalPerDataPunt], ignore_index= True)
                            MatrixPerDataPunt = pd.concat([ConfusionMatrix, MatrixPerDataPunt], ignore_index= True)
                            
                        pbar.update(1)
                    
# Slaat alle resultaten op in de output Excel
OutputFolder = Path(OutputFolder)
if Clustering and ScheidingsTekenVariabelen:
    with pd.ExcelWriter(os.path.join(OutputFolder, f"AAA_Resultaten_{OutputFolder.stem}.xlsx")) as Writer:
        TotaalPerDataPunt.to_excel(Writer, sheet_name= "Output per data punt", index=False)
        MatrixPerDataPunt.to_excel(Writer, sheet_name= "Matrix per data punt", index=False)
        TotaalPerCluster.to_excel(Writer, sheet_name= "Output per clusters", index=False)
        MatrixPerCluster.to_excel(Writer, sheet_name= "Matrix per clusters", index=False)
else:
    with pd.ExcelWriter(os.path.join(OutputFolder, f"AAA_Resultaten_{OutputFolder.stem}.xlsx")) as Writer:
        TotaalPerDataPunt.to_excel(Writer, sheet_name= "Output per data punt", index=False)
        MatrixPerDataPunt.to_excel(Writer, sheet_name= "Matrix per data punt", index=False)